In [1]:
!pip install transformers datasets torch

In [31]:
import torch
from datasets import load_dataset
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [32]:
# Load dataset (2% of the training set)
dataset = load_dataset('amazon_polarity', split='train[:1%]')

# Split dataset into train and test (e.g., 90% train, 10% test)
split = dataset.train_test_split(test_size=0.1)
train_dataset = split['train']
test_dataset = split['test']


In [33]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['content'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=32)
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=32)

# Set formats for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/32400 [00:00<?, ? examples/s]

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

In [34]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}


In [47]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    report_to=[]  # Disable all integrations including wandb
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [48]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.315100,0.220217,0.925000,0.926511,0.935165,0.918015
2,0.193700,0.301967,0.929167,0.931062,0.933333,0.928803


TrainOutput(global_step=8100, training_loss=0.23235297991905685, metrics={'train_runtime': 1733.3388, 'train_samples_per_second': 37.384, 'train_steps_per_second': 4.673, 'total_flos': 4262399096832000.0, 'train_loss': 0.23235297991905685, 'epoch': 2.0})

In [49]:
eval_results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 0.3019665479660034, 'eval_accuracy': 0.9291666666666667, 'eval_f1': 0.9310624493106245, 'eval_precision': 0.9333333333333333, 'eval_recall': 0.9288025889967637, 'eval_runtime': 26.4878, 'eval_samples_per_second': 135.911, 'eval_steps_per_second': 8.494, 'epoch': 2.0}


In [50]:
model.save_pretrained('./fine_tuned_bert_amazon')
tokenizer.save_pretrained('./fine_tuned_bert_amazon')


('./fine_tuned_bert_amazon/tokenizer_config.json',
 './fine_tuned_bert_amazon/special_tokens_map.json',
 './fine_tuned_bert_amazon/vocab.txt',
 './fine_tuned_bert_amazon/added_tokens.json',
 './fine_tuned_bert_amazon/tokenizer.json')